In [ ]:
import llama_cpp
from PIL import Image
import requests
from pathlib import Path

def download_model_files(base_url, model_files):
    """Download model files if they don't exist"""
    for filename in model_files:
        if not Path(filename).exists():
            url = f"{base_url}/resolve/main/{filename}"
            response = requests.get(url)
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"Downloaded {filename}")

def load_image(image_path):
    """Load and return a PIL Image"""
    if image_path.startswith(('http://', 'https://')):
        return Image.open(requests.get(image_path, stream=True).raw)
    return Image.open(image_path)

def caption_image(image_path, model_path="llava-llama-3-8b-v1_1-f16.gguf", 
                 mmproj_path="llava-llama-3-8b-v1_1-mmproj-f16.gguf"):
    """Generate a caption for the given image using LLaVA model"""
    
    # Initialize the LLaVA model
    model = llama_cpp.LlamaVision(
        model_path=model_path,
        mmproj_path=mmproj_path,
        n_ctx=4096,
        verbose=False
    )
    
    # Load the image
    image = load_image(image_path)
    
    # Create prompt
    prompt = """<|start_header_id|>user<|end_header_id|>

<image>
Describe this image<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    # Generate caption
    response = model.create_chat_completion(
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "image", "source": image},
                    {"type": "text", "text": "Describe this image"}
                ]
            }
        ],
        max_tokens=512,
        temperature=0.7
    )
    
    return response['choices'][0]['message']['content']

# Example usage
if __name__ == "__main__":
    # Model files to download
    base_url = "https://huggingface.co/xtuner/llava-llama-3-8b-v1_1-gguf"
    model_files = [
        "llava-llama-3-8b-v1_1-f16.gguf",
        "llava-llama-3-8b-v1_1-mmproj-f16.gguf"
    ]
    
    # Download models if needed
    download_model_files(base_url, model_files)
    
    # Test with an image
    image_path = "https://i.pinimg.com/736x/ab/4a/a1/ab4aa1119864bbae52fa8a99baf3f71e.jpg"  # Replace with your image path
    caption = caption_image(image_path)
    print("Generated Caption:", caption)